In [2]:
import numpy as np

from typing import Any, Optional, Tuple

import torch
from torch import Tensor

from ax.modelbridge.factory import get_GPEI, get_botorch
from ax.modelbridge.registry import Models, TorchModelBridge
from ax.models.torch.botorch import BotorchModel

from botorch.sampling import SobolQMCNormalSampler
from botorch.models.model import Model
from botorch.acquisition.acquisition import AcquisitionFunction
from botorch.acquisition import get_acquisition_function

from botorch.utils import (
    get_objective_weights_transform,
    get_outcome_constraint_transforms
)

from ax import (
    Data,
    Metric,
    Experiment,
    Objective,
    OutcomeConstraint,
    OptimizationConfig,
    ComparisonOp,
    ParameterType, 
    RangeParameter,
    SearchSpace, 
    SimpleExperiment, 
    OutcomeConstraint, 
)

from albo.acquisition.objective import ClassicAugmentedLagrangianMCObjective
from albo.optim.optimize import optimize_al_inner

In [3]:
class AlboEiConstructor(object):
    
    def __init__(self, objective_constructor, bounds, init_mults=None, rate=1.0, niter=10):
        self.objective_constructor = objective_constructor
        self.bounds = bounds
        self.init_mults = init_mults
        self.rate = rate
        self.niter = niter
    
        self.x = None
        self.L = None
        self.trace = None
        self.mults = None
    
    def __call__(self,
        model: Model,
        objective_weights: Tensor,
        outcome_constraints: Optional[Tuple[Tensor, Tensor]] = None,
        X_observed: Optional[Tensor] = None,
        X_pending: Optional[Tensor] = None,
        **kwargs: Any 
    ) -> AcquisitionFunction:
    
        sampler = SobolQMCNormalSampler(num_samples=512)   
    
        objective = self.objective_constructor(
            objective=get_objective_weights_transform(objective_weights),
            constraints=get_outcome_constraint_transforms(outcome_constraints),
            r = self.rate,
            mults=self.init_mults
        )
        
        self.x, self.L, self.trace = optimize_al_inner(
            model=model,
            objective=objective,
            sampler=sampler,
            bounds=self.bounds,
            niter=self.niter
        )
           
        self.mults = objective.mults
            
        return get_acquisition_function(
            acquisition_function_name="qEI",
            model=model,
            objective=objective,
            X_observed=X_observed,
            X_pending=X_pending,
            prune_baseline=kwargs.get("prune_baseline", True),
            mc_samples=kwargs.get("mc_samples", 512),
            qmc=kwargs.get("qmc", True),
            seed=torch.randint(1, 10000, (1,)).item(),
        )            

In [4]:
def gramacy_evaluation_fcn(
    parametrization, # Mapping of parameter names to values of those parameters.
    weight=None, # Optional weight argument.
):
    x1 = parametrization.get('x')
    x2 = parametrization.get('y')
    
    f = x1 + x2
    c1 = (3./2) - x1 - 2.*x2 - (1./2) * np.sin(2. * np.pi * (x1**2 - 2.*x2))
    c2 = x1**2 + x2**2 - 3./2
    
    return {
        'f': (f, 0.0), 
        'c1': (c1, 0.0), 
        'c2': (c2, 0.0)
    }

def get_evaluations(arms):
    evaluations = {}
    for arm in arms:
        evaluations[arm.name] = gramacy_evaluation_fcn(arm.parameters)
    return evaluations

In [5]:
search_space = SearchSpace(
    parameters=[
        RangeParameter(name='x', parameter_type=ParameterType.FLOAT, lower=0.0, upper=1.0),
        RangeParameter(name='y', parameter_type=ParameterType.FLOAT, lower=0.0, upper=1.0)
    ]
)

objective = Objective(metric=Metric(name='f'), minimize=True)
constraint1 = OutcomeConstraint(metric=Metric(name='c1'), op=ComparisonOp.LEQ, bound=0.0, relative=False)
constraint2 = OutcomeConstraint(metric=Metric(name='c2'), op=ComparisonOp.LEQ, bound=0.0, relative=False)

optimization_config = OptimizationConfig(
    objective=objective, 
    outcome_constraints=[
        constraint1, 
        constraint2
    ]
)

experiment = Experiment(
    name='gramacy',
    search_space=search_space,
    optimization_config=optimization_config
)

In [6]:
sobol = Models.SOBOL(search_space=search_space)
gr = sobol.gen(n=5)
trial = experiment.new_batch_trial()
trial.add_arms_and_weights(gr.arms)
trial.mark_running(no_runner_required=True)

evaluations = get_evaluations(trial.arms)
data = Data.from_evaluations(evaluations, trial.index)
experiment.attach_data(data)
trial.mark_completed()

BatchTrial(experiment_name='gramacy', index=0, status=TrialStatus.COMPLETED)

In [7]:
augmenter = ClassicAugmentedLagrangianMCObjective

for i in range(25):
    acqf_constructor = AlboEiConstructor(
        objective_constructor=ClassicAugmentedLagrangianMCObjective,
        bounds=torch.tensor([[0.0, 0.0], [1.0, 1.0]], dtype=torch.double),
        rate=10.0
    )

    model = get_botorch(
        experiment=experiment,
        search_space=search_space,
        data=experiment.fetch_data(),
        acqf_constructor=acqf_constructor
    )

    generator_run = model.gen(1)
    trial = experiment.new_batch_trial(generator_run=generator_run)
    trial.mark_running(no_runner_required=True)
    
    evaluations = get_evaluations(trial.arms)
    data = Data.from_evaluations(evaluations, trial.index)
    experiment.attach_data(data)
    trial.mark_completed()
    
    # proposed soludtion
    print('Iteration %d, Estimated optimal point %s' % (i, acqf_constructor.x.numpy()))

Iteration 0, Estimated optimal point [[[0.44897616 0.38944459]]]
Iteration 1, Estimated optimal point [[[0.44844288 0.40188728]]]
Iteration 2, Estimated optimal point [[[0.38109318 0.43058456]]]
Iteration 3, Estimated optimal point [[[0.33724582 0.46922055]]]
Iteration 4, Estimated optimal point [[[0.29383973 0.4541269 ]]]
Iteration 5, Estimated optimal point [[[0.21095629 0.42104085]]]
Iteration 6, Estimated optimal point [[[0.21910962 0.4225449 ]]]
Iteration 7, Estimated optimal point [[[0.19190009 0.40966486]]]
Iteration 8, Estimated optimal point [[[0.19208566 0.40939073]]]
Iteration 9, Estimated optimal point [[[0.19138494 0.40963595]]]
Iteration 10, Estimated optimal point [[[0.19469444 0.40497343]]]
Iteration 11, Estimated optimal point [[[0.19514489 0.40464247]]]
Iteration 12, Estimated optimal point [[[0.19514232 0.40464305]]]
Iteration 13, Estimated optimal point [[[0.19514897 0.40463281]]]
Iteration 14, Estimated optimal point [[[0.19514913 0.40463166]]]
Iteration 15, Estima